Ran LDA model on the first 1000 words of each podcast episode transcript with 200 topics, using Mallet. Each episode is then represented as its topic distribution. Depicted using a sample of 25K episodes, colored by category, and projected using t-SNE on episodes’ topic distributions to visualize topical distance, with select topic clusters annotated using the top words in the corresponding topic.

In [2]:
import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect('data.db')
cursor = conn.cursor()

# Fetch the first 10 entries from podcast_episodes
query = "SELECT rowid, epDescription, tokens FROM podcast_episodes LIMIT 10"
cursor.execute(query)
episodes = cursor.fetchall()

# Print the first 10 entries
for episode in episodes:
    print(episode)

# Close the connection
conn.close()

(1, '<p>Best of with snippets from 3 episodes and acoustic performance of 3 new songs.</p>', 'best snippets episodes acoustic performance new songs')
(2, '<p>Simon introduces &apos;It&apos;s all Gone&apos;, track 4 of the newly released KisTone Album, &apos;Way To Nowhere&apos;.</p>', 'simon introduces apositaposs goneapos track newly released kistone album aposway nowhereapos')
(3, '<p>Simon introduces track 4, of the newly released\xa0 KisTone album,\xa0 &apos;Today is Yesterday&apos;.</p>', 'simon introduces track newly released kistone album apostoday yesterdayapos')
(4, '<p>This week, Simon introduces track 2- Saturn Return of the KisTone album, &apos;Way To Nowhere&apos; and describes the background and motivation in co-writing the song \xa0with fellow band mates.</p>', 'week simon introduces track saturn return kistone album aposway nowhereapos describes background motivation cowriting song fellow band mates')
(5, '<p>Big news week. The band Simon lived in the USA with, KisTone 

In [2]:
import pandas as pd

In [4]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('data.db')

query = "SELECT epDescription, category1, category2, category3, category4, category5, category6, category7, category8, category9, category10 FROM podcast_episodes WHERE epDescription IS NOT NULL"
df = pd.read_sql_query(query, conn)
conn.close()

print(f"Loaded {len(df)} episode descriptions.")

Loaded 1124058 episode descriptions.


In [5]:
df.head()

,epDescription,category1,category2,category3,category4,category5,category6,category7,category8,category9,category10
0,<p>Best of with snippets from 3 episodes and a...,music,society,culture,philosophy,education,self improvement,NULL,NULL,NULL,NULL
1,<p>Simon introduces &apos;It&apos;s all Gone&a...,music,society,culture,philosophy,education,self improvement,NULL,NULL,NULL,NULL
2,"<p>Simon introduces track 4, of the newly rele...",music,society,culture,philosophy,education,self improvement,NULL,NULL,NULL,NULL
3,"<p>This week, Simon introduces track 2- Saturn...",music,society,culture,philosophy,education,self improvement,NULL,NULL,NULL,NULL
4,<p>Big news week. The band Simon lived in the ...,music,society,culture,philosophy,education,self improvement,NULL,NULL,NULL,NULL


In [ ]:
# import nltk

# nltk_data_path = "../data/nltk_data"
# nltk.data.path.append(nltk_data_path)

# nltk.download('punkt', download_dir=nltk_data_path)
# nltk.download('stopwords', download_dir=nltk_data_path)

In [8]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

stop_words = set(stopwords.words('english'))

text = "banana"
tokens = word_tokenize(text)

# def preprocess_text(text):
#     # Remove non-alphanumeric characters and lowercase
#     text = re.sub(r'[^a-zA-Z\s]', '', text.lower())
#     # Tokenize and remove stopwords
#     tokens = word_tokenize(text)
#     return [word for word in tokens if word not in stop_words and len(word) > 2]

# # Apply preprocessing
# df['tokens'] = df['epDescription'].apply(preprocess_text)

LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - '/cephyr/users/theale/Alvis/nltk_data'
    - '/mimer/NOBACKUP/groups/naiss2024-22-185/theodora/nltk_env/nltk_data'
    - '/mimer/NOBACKUP/groups/naiss2024-22-185/theodora/nltk_env/share/nltk_data'
    - '/mimer/NOBACKUP/groups/naiss2024-22-185/theodora/nltk_env/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


In [ ]:
from gensim import corpora, models

# Create a dictionary and corpus
dictionary = corpora.Dictionary(df['tokens'])
corpus = [dictionary.doc2bow(text) for text in df['tokens']]

# Train the LDA model
num_topics = 200
lda_model = models.LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=10)

# Get topic distributions for each episode
episode_topics = [lda_model.get_document_topics(bow) for bow in corpus]

# Convert to dense representation for visualization
import numpy as np
dense_topics = np.zeros((len(episode_topics), num_topics))
for i, topic_dist in enumerate(episode_topics):
    for topic_id, topic_value in topic_dist:
        dense_topics[i, topic_id] = topic_value

In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

# Run t-SNE
tsne = TSNE(n_components=2, perplexity=30, random_state=42)
tsne_results = tsne.fit_transform(dense_topics)

# Plot results with colors by category
categories = df['category1'].fillna('Unknown')

plt.figure(figsize=(12, 8))
scatter = plt.scatter(tsne_results[:, 0], tsne_results[:, 1], c=pd.factorize(categories)[0], cmap='tab20')
plt.legend(handles=scatter.legend_elements()[0], labels=pd.factorize(categories)[1])
plt.title("t-SNE Visualization of Podcast Topics")
plt.show()

In [ ]:
# Print top words for each topic
for i in range(num_topics):
    words = lda_model.show_topic(i, topn=10)
    top_words = ", ".join([word for word, _ in words])
    print(f"Topic {i}: {top_words}")